In [2]:
from PIL import Image, ImageDraw, ImageFont
import os

# Configuration
input_folder = "sankey_new"
output_file = "sankey/combined_Mauro.png"
images_per_row = 2
label_height = 50  # Increased space for label
font_size = 40
background_color = "#ffffff"

# Get all PNG files
image_files = [f for f in os.listdir(input_folder) 
              if f.lower().endswith('.png') and f.lower().startswith('sankey_mauro') and not f.lower().endswith('scgae.png')]
image_files.sort()

# Load font (make sure this path is correct)
try:
    font = ImageFont.truetype("arial.ttf", font_size)
except:
    font = ImageFont.load_default()
    print("Using default font - consider installing arial.ttf for better results")

images = []
for f in image_files:
    img = Image.open(os.path.join(input_folder, f)).convert("RGBA")
    label = os.path.splitext(f)[0].replace("sankey_Mauro_human_Pancreas_cell_", "")  # Clean filename for label
    images.append((img, label))

widths, heights = zip(*(i[0].size for i in images))

# Calculate dimensions
rows = (len(images) + images_per_row - 1) // images_per_row
total_width = max(widths) * images_per_row
max_height = (max(heights) + label_height) * rows  # Make sure we have space for labels

new_image = Image.new('RGBA', (total_width, max_height), background_color)
draw = ImageDraw.Draw(new_image)

x_offset, y_offset = 0, 0
for i, (im, label) in enumerate(images):
    # Paste the image first
    new_image.paste(im, (x_offset, y_offset), im)
    
    # Draw label BELOW the image (fixed position)
    text_width = draw.textlength(label, font=font)
    text_x = x_offset + (im.width - text_width) // 2  # Center in column
    text_y = y_offset + im.height + 5  # 5 pixels below image
    
    # Draw white background for text (optional, helps readability)
    draw.rectangle(
        [text_x - 2, text_y - 2, 
         text_x + text_width + 2, text_y + font_size + 2],
        fill=background_color
    )
    
    draw.text((text_x, text_y), label, fill="black", font=font)
    
    # Update offsets
    x_offset += im.width
    if (i + 1) % images_per_row == 0:
        x_offset = 0
        y_offset += im.height + label_height  # Move down by image + label space

new_image.save(output_file)
print(f"Saved labeled image to {output_file}")
print(f"Labels used: {[label for _, label in images]}")

Saved labeled image to sankey/combined_Mauro.png
Labels used: ['AttentionAE-sc', 'DEC', 'DESC', 'scCDCG', 'scDCC', 'scDSC', 'scDeepCluster', 'scGNN', 'scMAE', 'scNAME', 'scziDesk']
